In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen= ImageDataGenerator(rescale= 1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
training_set = train_datagen.flow_from_directory('/content/drive/My Drive/braintrain', target_size=(224,224),batch_size=32,class_mode='categorical',shuffle=True)
test_datagen= ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/braintest', target_size=(224,224),batch_size=32,class_mode='categorical')

Found 210 images belonging to 2 classes.
Found 43 images belonging to 2 classes.


Loading the DenseNet Model

In [0]:
import tensorflow as tf
loaded_model = tf.keras.models.load_model('/content/drive/My Drive/brainmodel.h5')

In [4]:
import numpy as np
from tensorflow_model_optimization.sparsity import keras as sparsity
epochs = 4
end_step = np.ceil(1.0 * len(training_set)).astype(np.int32) * epochs
print(end_step)
 
#Defining parameters for pruning 
#Sparsity is taken 50% for initial and 90% at final 
new_pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                   final_sparsity=0.90,
                                                   begin_step=0,
                                                   end_step=end_step,
                                                   frequency=100)
}

new_pruned_model = sparsity.prune_low_magnitude(loaded_model, **new_pruning_params)

#getting summary of the model
new_pruned_model.summary()

#compiling pruned model
new_pruned_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

28
Instructions for updating:
Please use `layer.add_weight` method instead.

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
prune_low_magnitude_zero_paddin (None, 230, 230, 3)  1           input_1[0][0]                    
__________________________________________________________________________________________________
prune_low_magnitude_conv1/conv  (None, 112, 112, 64) 18818       prune_low_magnitude_zero_padding2
__________________________________________________________________________________________________
prune_low_magnitude_conv1/bn (P (None

Defining callback and Fitting the Pruned model

In [5]:
callbacks = [
    sparsity.UpdatePruningStep(),
    sparsity.PruningSummaries(log_dir='logdir', profile_batch=0)
]

new_pruned_model.fit(training_set,
          epochs=epochs,
          verbose=1,
          callbacks=callbacks,
          validation_data=test_set)

score = new_pruned_model.evaluate(test_set, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/4
7/7 [==============================] - 177s 25s/step - loss: 0.5710 - acc: 0.7095 - val_loss: 0.3550 - val_acc: 0.8140
Epoch 2/4
7/7 [==============================] - 110s 16s/step - loss: 0.4908 - acc: 0.7524 - val_loss: 0.3127 - val_acc: 0.8605
Epoch 3/4
7/7 [==============================] - 109s 16s/step - loss: 0.5383 - acc: 0.7381 - val_loss: 0.2937 - val_acc: 0.8837
Epoch 4/4
7/7 [==============================] - 110s 16s/step - loss: 0.4485 - acc: 0.8000 - val_loss: 0.3060 - val_acc: 0.8372
Test loss: 0.26695140451192856
Test accuracy: 0.8604651


Accuracy after prunning : 86.04%

In [6]:
final_model = sparsity.strip_pruning(new_pruned_model)
final_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[1][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[1][0]                 
______________________________________________________________________________________________

Saving the Prunedmodel in .h5 format

In [0]:
tf.keras.models.save_model(final_model, 'prunedmodel.h5', 
                        include_optimizer=False)

Converting the .h5 model into .tflite format

In [19]:
tflite_model_file = 'prunedmodel.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model_file('prunedmodel.h5')
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()
with open(tflite_model_file, 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Froze 1006 variables.
INFO:tensorflow:Converted 1006 variables to const ops.


In [20]:
print("Size of the pruned model after conversion to tflite format : %.2f Mb" 
      % (os.path.getsize('prunedmodel.tflite')/ float(2**20)))

Size of the pruned model after conversion to tflite format : 18.00 Mb


In [21]:
!ls -lh *.tflite

-rw-r--r-- 1 root root 18M Mar 25 15:41 prunedmodel.tflite


As seen size of the pruned model has decreased drastically to 18 Mb